## Filter
One of the most powerful parameters when defining a search is the ability to filter the results.  While a Query is more of an unstructured approach to search for text within documents, a Filter provides a well-defined structured approach when we know which properties we want to restrict our matching documents.

The following examples will be using some of the popular properties as outlined within the "Properties & Metadata" notebook.  In addition, the selection of these properties was determined based on the discovery techniques used such as metadata and debugging.

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd
import datetime as dt
import dateutil.relativedelta

# Default session - desktop
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f9e9ef19820 {name='codebook'}>

In [2]:
# Uncomment the following to see a larger data set when displaying to the screen.
pd.set_option('display.max_colwidth', 140)
#pd.set_option('display.max_rows', 10000)

rd.__version__

'1.0.0b10'

#### Example - Query vs Filter
As you become more familiar with Search, you will realize there are a number of different paths you can take to arrive at the same destination.

In [3]:
# Retrieve the document(s) for a specific ISIN (Astra Zeneca) using a Query.
response=search.Definition(
    view = search.SearchViews.INSTRUMENTS,
    query = 'US046353AQ14',
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
).get_data()
response.data.df

,DocumentTitle,RIC,ISIN,CUSIP,SEDOL,PermID
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [4]:
# Retrieve the document(s) for the same ISIN using a Filter
response=search.Definition(
    view = search.SearchViews.INSTRUMENTS,
    filter = "ISIN eq 'US046353AQ14'",
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
).get_data()
response.data.df

,DocumentTitle,RIC,ISIN,CUSIP,SEDOL,PermID
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


#### Example - Apple Equities

In [5]:
# Within the view (EquityQuotes), select all documents where the term 'AAPL' is within the ticker symbol.  For each, 
# ensure the type of asset is an 'equity' and that each asset is in an active state.  I chose to filter out the 
# AssetState of 'DC' (De-activated) as my criteria. By default, the top 10 results are presented.
#
# Note: An alternative to "AssetType eq 'equity'" could be to use: "RCSAssetCategoryLeaf eq 'Ordinary Share'"
response=search.Definition(
    view = search.SearchViews.EQUITY_QUOTES,
    filter = "AssetType eq 'equity' and AssetState ne 'DC' and TickerSymbol eq 'AAPL'",
    select = "RIC, CompositeDescriptiveName, ExchangeCode, ExchangeName, AssetType"
).get_data()
response.data.df

,RIC,CompositeDescriptiveName,ExchangeCode,ExchangeName,AssetType
0,AAPL.O,Apple Ord Shs APPLE ORD,NSQ,NASDAQ Global Select Consolidated,[EQUITY]
1,AAPLEUR.xbo,Apple Ord Shs APPLE ORD APPLE INC,XBO,MiFID Composite Cross Market Service,[EQUITY]
2,AAPL.OQ,Apple Ord Shs APPLE ORD,NSM,NASDAQ Stock Exchange Global Select Market,[EQUITY]
3,AAPL.DG,Apple Ord Shs APPLE ORD,GCD,Cboe EDGX Exchange - Nasdaq Global Select Market,[EQUITY]
4,AAPL.Z,Apple Ord Shs APPLE ORD APPLE INC,GCZ,Cboe BZX Exchange - Nasdaq Global Select Market,[EQUITY]
5,AAPL.B,Apple Ord Shs APPLE ORD,GSB,Boston SE when trading NASDAQ Global Select Market,[EQUITY]
6,AAPLE.MI,Apple Ord Shs APPLE ORD,MIL,Milan Stock Exchange,[EQUITY]
7,AAPL.CCP,Apple 100 CDR Representing 12 Ord Shs APPLE 100 CDR REP 12 ORD,CAQ,Canadian Composite Quote/Trade,[EQUITY]
8,AAPL.DY,Apple Ord Shs APPLE ORD,GDA,Cboe EDGA Exchange - Nasdaq Global Select Market,[EQUITY]
9,AAPL.PH,Apple Ord Shs APPLE ORD APPLE INC,GSX,NASDAQ OMX PHLX Global Select Market,[EQUITY]


In [6]:
response.total

55

#### Example - Vessels
List all Very Large Gas Containers (VLGC) that are in service within the last week.

In [7]:
# Build date range expression for the location of the vessel for the last 7 days
date_7_days_ago = dt.datetime.today() - dateutil.relativedelta.relativedelta(days=7)
date_range = f'LocationTimestamp ge {date_7_days_ago.strftime("%Y-%m-%d")} and \
               LocationTimestamp lt {dt.datetime.today().strftime("%Y-%m-%d")}'

In [8]:
# Show the location of each vessel and some general details such as location, destination and coordinates.
# If available, show the top 100 hits.
response=search.Definition(
    view = search.SearchViews.VESSEL_PHYSICAL_ASSETS,
    filter = f"RCSAssetTypeLeaf eq 'VLGC' and VesselStatus eq 'IN SERVICE' and {date_range}",
    select = 'RIC, SynonymSearchAll, DTSource, LocationTimestamp, IMO,Latitude, Longitude, LocationDestination'
).get_data()
response.data.df

,RIC,SynonymSearchAll,DTSource,LocationTimestamp,IMO,Latitude,Longitude,LocationDestination
0,C}KY7309989243,Chinese Liquefied Petroleum Gas Very Large Gas Carrier,Shanghai|East China Sea|Shanghai Tanker Zone|China (Mainland),2022-04-19T05:50:27.000Z,9901403,31.353405,121.729019,JIANG NAN SHIPYARD
1,C}KY7309461813,Indonesian India Liquefied Petroleum Gas Very Large Gas Carrier,Indonesia|Java Sea|Singapore Tanker Zone,2022-04-21T07:31:10.000Z,8910897,-0.908895,104.101067,TG. JABUNG
2,C}KY7309473669,Liquefied Petroleum Gas Very Large Gas Carrier Korean Chinese,Java Sea|Singapore Tanker Zone,2022-04-21T02:04:23.000Z,9114581,1.352045,104.406631,"LIANYUNGAN,CHINA [Lianyungang]"
3,C}KL7309478067,Chinese Japanese Liquefied Petroleum Gas Very Large Gas Carrier Korean,East China Sea|Taiwan Tanker Zone,2022-04-21T06:54:05.000Z,9172636,24.645056,122.708214,CNLKU [Longkou]
4,C}KY7309636853,Liquefied Petroleum Gas Very Large Gas Carrier,Red Sea|Yanbu Tanker Zone,2022-04-15T18:36:05.000Z,9719496,17.057825,42.115723,JIZAN [Gizan Marine Port]
5,C}KY7309843619,Liquefied Petroleum Gas Very Large Gas Carrier Japanese,Pacific Ocean (Central America)|Panama Tanker Zone,2022-04-16T17:14:37.000Z,9749788,10.944246,-93.079643,JP CHB [Chiba]
6,C}KY7309466419,Liquefied Petroleum Gas Very Large Gas Carrier,Gulf of Oman|Arabian Gulf Tanker Zone,2022-04-15T23:31:38.000Z,9000883,25.661646,54.992416,SHARJAH [Mina Khalid (Sharjah)]
7,C}KY7309479648,Indonesian India Liquefied Petroleum Gas Very Large Gas Carrier,Indonesia|East Kalimantan|Java Sea|Lawi-Lawi Terminal Tanker Zone,2022-04-21T17:01:08.000Z,9193733,-1.325167,116.97551,BALIKPAPAN [Balikpapan]
8,C}KY7309486846,Malaysian Liquefied Petroleum Gas Very Large Gas Carrier,Java Sea|Singapore Tanker Zone|Singapore Strait Waypoint|Malaysia,2022-04-21T03:59:16.000Z,9290270,1.45157,103.878258,MYPGU [Pasir Gudang]


#### Example - Floating Rate Notes
Retrieve all floating rate notes (floating coupon) in the healthcare sector which are issued in Euros.  For each, ensure the coupon rate is great than 2 and our Moody's rating is greater than 'Baa1'.

To determine the rating, we apply the following logic utilizing the RatingsScope property - which is a nested property providing additional details:

- RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10)

The RatingType of 'MIS' refers to the "Moody's Long-term Issuer Rating".
The CurrentRatingRank < 10 instructs search to look for ratings less than 10 ('Baa1').  **Note**: The lower the rank, the better the rating.

In [9]:
response=search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    filter = "CouponRate gt 2 and \
              RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10) and \
              RCSCouponTypeName eq 'Floating Coupon' and \
              IsActive eq true and \
              RCSTRBC2012Name eq 'Healthcare' and \
              RCSCurrencyLeaf xeq 'Euro'",
    select = "CouponCurrency, IssuerName, IssueDate, RIC, CouponRate, ISIN, Ticker, RCSTRBC2012Leaf, MaturityDate, AssetState",   
).get_data()
response.data.df

,CouponCurrency,IssuerName,IssueDate,RIC,CouponRate,ISIN,Ticker,RCSTRBC2012Leaf
0,EUR,Sanofi FR,1983-07-01T00:00:00.000Z,FR0000140022=,15.5,FR0000140022,SASY,Pharmaceuticals (NEC)
1,EUR,Sanofi FR,1987-10-01T00:00:00.000Z,FR0000047656=AURO,15.5,FR0000047656,SASY,Pharmaceuticals (NEC)


#### Example - Outstanding Green Bonds
List the outstanding Green Bonds for Telefonica.

In [10]:
response=search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    filter = "IssuerName eq 'Telefonica' and IsGreenBond eq true",
    select = "IssuerName, DocumentTitle, RIC, CouponRate, ISIN, IssuerTicker, DebtTypeDescription, EOMAmountOutstanding, FaceOutstanding"
).get_data()
response.data.df

,IssuerName,DocumentTitle,RIC,CouponRate,ISIN,IssuerTicker,DebtTypeDescription,EOMAmountOutstanding,FaceOutstanding
0,Telefonica Europ,"Telefonica Europe BV, Bond, TEF 2.502",ES210981985=,2.502,XS2109819859,TEFBV,Junior Subordinated Note,500000000,500000000
1,Telefonica Emis,"Telefonica Emisiones SAU, Plain Vanilla Fixed Coupon Bond, TEF 1.069 05-Feb-2024",ES194600445=,1.069,XS1946004451,TEFEMI,Senior Note,1000000000,1000000000


#### Example - Top executives

In [11]:
# Using the People View, we can utilize the OfficerDirector property, which contains a number of nested or sub-properties 
# that allow me to perform a more fine-grained search against active people that have been assigned a title code of 
# CEO (Chief Executive Officer). The results are listed based on a default rank defined by the search service.
response=search.Definition(
    view = search.SearchViews.PEOPLE,
    filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')",
    top = 20,
    select = "FullName, PrimaryTitleAsReported, PrimaryOfficerDirector, RIC, OfficerDirectorActiveCompanies, PrimaryOfficerDirectorRIC"
).get_data()
response.data.df

,FullName,PrimaryTitleAsReported,PrimaryOfficerDirector,OfficerDirectorActiveCompanies,PrimaryOfficerDirectorRIC
0,Satya Nadella,"Chairman of the Board, Chief Executive Officer",Microsoft Corp,"[Starbucks Corp, Microsoft Deutschland GmbH, Chandler Insurance Company Ltd, Microsoft Corp]",MSFT.O
1,Tim D. Cook,"Chief Executive Officer, Director",Apple Inc,"[Apple Inc, Nike Inc, Apple Retail Uk Ltd]",AAPL.O
2,Andrew R. Jassy,"President, Chief Executive Officer, Director",Amazon.com Inc,"[Coupa Software Inc, Amazon.com Inc]",AMZN.O
3,Sundar Pichai,"Chief Executive Officer, Director",Alphabet Inc,"[Alphabet Inc, Ruba Inc (Fka Kudo Inc)]",GOOGL.O
4,Mark Zuckerberg,"Chairman of the Board, Chief Executive Officer, Founder",Meta Platforms Inc,"[Chan Zuckerberg Initiative LLC, Meta Platforms Inc]",FB.O
5,Warren E. Buffett,"Chairman of the Board, Chief Executive Officer",Berkshire Hathaway Inc,"[Grinnell College, Bill & Melinda Gates Foundation Trust, Berkshire Hathaway Inc, Kraft Heinz Intermediate Corporation II, Kraft Heinz F...",BRKa
6,Daniel Zhang,"Chairman of the Board, Chief Executive Officer",Alibaba Group Holding Ltd,"[Alibaba Group Holding Ltd, DiDi Global Inc]",BABA.K
7,Fan Jiang,President - Taobao and Tmall,Alibaba Group Holding Ltd,"[Alibaba Group Holding Ltd, Umeng Technology Ltd]",BABA.K
8,Weijian Shan,Independent Director,Alibaba Group Holding Ltd,"[Wilmar Ventures Pte Ltd, Alibaba Group Holding Ltd, TCC International Holdings Ltd, Pacific Alliance Group Ltd, Storybook Entertainment...",BABA.K
9,Huateng Ma,"Executive Chairman of the Board, Chief Executive Officer",Tencent Holdings Ltd,"[Tencent Holdings Ltd, Tencent Technology Beijing Co Ltd]",0700.HK


In [12]:
rd.close_session()